In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from utils import session_config
os.environ['CUDA_VISIBLE_DEVICES'] = "0" # 사용 gpu 선택
session_config.setup_gpus(True, 0.25) # gpu 메모리 사용량 세팅

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import tensorflow as tf

# load your model already built at phase 1.
model = tf.keras.models.load_model("models/cifar_classifier.h5")

# load your dataset with tf.data
cifar10 = tf.keras.datasets.cifar10
(train_x, train_y), (val_x, val_y) = cifar10.load_data()
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
val_ds = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(64)
train_ds = train_ds.shuffle(buffer_size = 20000, reshuffle_each_iteration=False).batch(64).prefetch(tf.data.AUTOTUNE)

# Regular training phase

Phase 2 정규학습에서는 모델의 대략적인 성능을 확인하면서, 하이퍼파라미터 튜닝 이전에 주요한 요인들을 사전에 결정한다. 

정규학습에서 결정하는 주요한 하이퍼파라미터는 다음과 같다. 

- learning rate = [1E-3, 1E-4, 1E-5]
- learning rate scheduler = [constant, piecewise decay, linear decay, cosine decay restart]
- Initial parameter = [scratch, imagenet]

리스트에 포함된 파라미터값에 의해 총 3x4x2 = 24가지 경우의 수에 대해 실험할 수 있다. 각 실험에 대해 Early stopping 지점까지 학습하고 validation 정확도를 측정한다. 우리는 주요한 하이퍼파라미터를 Grid search 방식으로 사전에 결정한다. 이런 방식은 차후 하이퍼파라미터 튜닝에서 더 낮은 차원의 파라미터 공간에 대한 탐색을 수행할 수 있다. 이는 하이퍼파라미터 최적화에 대한 약간의 타협을 통해 효율적인 파라미터 튜닝을 가능하게 한다.

In [ ]:
from tensorboard.plugins.hparams import api as hp

#하이퍼파라미터 설정
HP_LR = hp.HParam('lr', hp.Discrete([1E-3, 1E-4, 1E-5]))
HP_LR_scheduler = hp.HParam('lr scheduler', hp.Discrete(['constant', 'piecewide decay',
                                                        'linear decay', 'cosine decay restart']))
HP_INIT = hp.HParam('init', hp.Discrete(['scratch', 'imagenet']))

hparams = [HP_LR, HP_LR_scheduler, HP_INIT]

#log 파일 생성
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_LR, HP_LR_scheduler, HP_INIT],
    metrics=[hp.Metric('accuracy', display_name='Accuracy')],
  )

def write_log(run_dir, hparams, accuracy):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    tf.summary.scalar('accuracy', accuracy, step=1)              

In [ ]:
def build_model():
    return tf.keras.models.load_model("models/cifar_classifier.h5")

@tf.function
def train_step(model, images, labels, optimizer):
    with tf.GradientTape() as tape:
        images = preprocessing(images)
        preds = model(images)
        loss = loss_fn(labels, preds)
        l2_loss = [tf.nn.l2_loss(t) for t in model.trainable_variables]
        l2_loss = l2_reg * tf.math.reduce_sum(l2_loss)
        total_loss += l2_loss
    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return total_loss, loss

@tf.function
def test_step(model, images, labels):
    images = preprocessing(images)
    preds = model(images)
    loss = loss_fn(labels, preds)
    return loss

def train_test(build_model, session_num, hparams):
    model = build_model()
    optimizer = tf.keras.optimizers.Adam(hparams["lr"])
    for epoch in range(EPOCHS):
        for x,y in train_ds:
            loss, train_loss, train_acc = train_step(model, x, y, optimizer)
        for x,y in val_ds:
            loss, test_loss, test_acc = test_step(model, x, y)

In [ ]:
session_num = 0
for init in HP_INIT.domain.values:
  for lr_scheduler in HP_LR_scheduler.domain.values:
    for lr in HP_LR.domain.values:
        hparams = {
          HP_LR.name.name: lr,
          HP_LR_scheduler.name: lr_scheduler,
          HP_INIT.name: init}
        accuracy = train_test(hparams)
        run_name = "run-%d" % session_num
        write_log('logs/hparam_tuning/' + run_name, hparams, accuracy)
        session_num += 1